***Code to calculate the before and after registration metrics on all tissues and plot box plots***

Last edited on 12/04/2024

The metrics csv file stored at: /nfs2/forGaurav/yesbackup/DAPI_reg/final_metrics

In [1]:
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from my_utils import Utils
import glob
import re
os.environ['NEURITE_BACKEND'] = 'pytorch'
os.environ['VXM_BACKEND'] = 'pytorch'
import voxelmorph as vxm
from my_utils import Utils
import torch
import csv

In [2]:
Image.MAX_IMAGE_PIXELS = None
def extract_round_number(filename):
    match = re.search(r'_ROUND_(\d+)_', filename)
    if match:
        return int(match.group(1))
    else:
        return 0
#Step 1: Load the images and apply the masks
tissue_names_file='/nfs2/baos1/rudravg/tissue_names.txt'
original_tissues_path='/nfs2/baos1/rudravg'

raw_dir='/home-local/rudravg/Segmentation_test/Images'
mask_dir='/home-local/rudravg/Segmentation_test/Masks'
model_path='/home-local/rudravg/test_DAPI/epochs/epoch_50/epoch_50.pth'

#Reading all tissue names
with open(tissue_names_file, 'r') as file:
    tissue_names = [line.strip() for line in file.readlines()]

In [3]:
with open('/nfs2/forGaurav/yesbackup/DAPI_reg/final_metricsoutput.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['tissue', 'round_x', 'before_ncc', 'after_ncc'])
    for tissue in tqdm(tissue_names,desc='Generating Images and Truths'):
        search_pattern_raw_files = os.path.join(original_tissues_path, tissue + '*')
        matching_files_raw = glob.glob(search_pattern_raw_files)
        sorted_files_raw = sorted(matching_files_raw, key=extract_round_number) #This is important
        mask_search_pattern = os.path.join(original_tissues_path, 'Retention_Masks', tissue + '*')
        mask_files = glob.glob(mask_search_pattern)
        round_0=sorted_files_raw[0]
        if mask_files:
            mask_name = mask_files[0]
            mask=np.array(Image.open(mask_name))
            mask = (mask > 0).astype(int)
        else:
            continue
        for round_x in sorted_files_raw[1:]:
            moving,fixed,orig_height,orig_width=Utils.load_images_and_apply_mask(round_0,round_x,mask_name)
            model,device=Utils.load_model(model_path)
            registered_tissue,L2_warp=Utils.register_tissues(moving,fixed,model,device)
            moving_unpadded=moving[:orig_height,:orig_width]
            fixed_unpadded=fixed[:orig_height,:orig_width]
            registered_tissue_unpadded=registered_tissue[:orig_height,:orig_width]
            L2_warp=L2_warp[:orig_height,:orig_width]
            before_ncc=Utils.calculate_ncc(fixed_unpadded.ravel(),moving_unpadded.ravel())
            after_ncc=Utils.calculate_ncc(fixed_unpadded.ravel(),registered_tissue_unpadded.ravel())
            writer.writerow([tissue, round_x, before_ncc, after_ncc])

Generating Images and Truths:   0%|          | 0/62 [00:00<?, ?it/s]

/home/local/VANDERBILT/rudravg/miniconda3/envs/dapi_reg/lib/python3.12/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Generating Images and Truths:  56%|█████▋    | 35/62 [31:57<25:04, 55.73s/it]  